In [390]:
import numpy as np
import pandas as pd
import pickle
#!pip install bz2file

import bz2file as bz2

In [356]:
movies=pd.read_csv("tmdb_5000_movies.csv")
credits=pd.read_csv("tmdb_5000_credits.csv")

In [357]:
movie_df=movies.merge(credits,on='title')

In [358]:
movie_df=movie_df[['id','title','overview','genres','keywords','cast','crew']]

In [359]:
#Null Data
movie_df[movie_df['overview'].isnull()]
movie_df.dropna(axis=0,inplace=True)

In [360]:
movie_df.isnull().sum()

id          0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
dtype: int64

In [361]:
movies.duplicated().sum()

0

In [362]:
def convert(val):
    ans=[i['name'] for i in eval(val)]
    return ans
movie_df['genres']=movie_df.genres.apply(convert)
movie_df['keywords']=movie_df.keywords.apply(convert)

In [363]:
def convert_cast(val):
    ans=[i['name'] for i in eval(val)]
    return ans[0:3]
movie_df['cast']=movie_df.cast.apply(convert_cast)

In [364]:
def convert_crew(val):
    return [x['name']for x in  eval(val) if x['job']=='Director']
movie_df['crew']=movie_df.crew.apply(convert_crew)

In [366]:
movie_df['overview']=movie_df.overview.apply(lambda x:x.split())

In [368]:
def collapse(L):
    L1 = []
    for i in L:
        L1.append(i.replace(" ",""))
    return L1

In [369]:
movie_df['cast'] = movie_df['cast'].apply(collapse)
movie_df['crew'] = movie_df['crew'].apply(collapse)
movie_df['genres'] = movie_df['genres'].apply(collapse)
movie_df['keywords'] = movie_df['keywords'].apply(collapse)

In [370]:
movie_df.head()

,id,title,overview,genres,keywords,cast,crew
0,19995,Avatar,"[In, the, 22nd, century,, a, paraplegic, Marin...","[Action, Adventure, Fantasy, ScienceFiction]","[cultureclash, future, spacewar, spacecolony, ...","[SamWorthington, ZoeSaldana, SigourneyWeaver]",[JamesCameron]
1,285,Pirates of the Caribbean: At World's End,"[Captain, Barbossa,, long, believed, to, be, d...","[Adventure, Fantasy, Action]","[ocean, drugabuse, exoticisland, eastindiatrad...","[JohnnyDepp, OrlandoBloom, KeiraKnightley]",[GoreVerbinski]
2,206647,Spectre,"[A, cryptic, message, from, Bond’s, past, send...","[Action, Adventure, Crime]","[spy, basedonnovel, secretagent, sequel, mi6, ...","[DanielCraig, ChristophWaltz, LéaSeydoux]",[SamMendes]
3,49026,The Dark Knight Rises,"[Following, the, death, of, District, Attorney...","[Action, Crime, Drama, Thriller]","[dccomics, crimefighter, terrorist, secretiden...","[ChristianBale, MichaelCaine, GaryOldman]",[ChristopherNolan]
4,49529,John Carter,"[John, Carter, is, a, war-weary,, former, mili...","[Action, Adventure, ScienceFiction]","[basedonnovel, mars, medallion, spacetravel, p...","[TaylorKitsch, LynnCollins, SamanthaMorton]",[AndrewStanton]


In [372]:
movie_df['tags'] = movie_df['overview'] + movie_df['genres'] + movie_df['keywords'] + movie_df['cast'] + movie_df['crew']


In [374]:
new_df = movie_df.drop(columns=['overview','genres','keywords','cast','crew'])


In [375]:
new_df['tags'] = new_df['tags'].apply(lambda x: " ".join(x))

In [376]:
new_df['tags'].head()

0    In the 22nd century, a paraplegic Marine is di...
1    Captain Barbossa, long believed to be dead, ha...
2    A cryptic message from Bond’s past sends him o...
3    Following the death of District Attorney Harve...
4    John Carter is a war-weary, former military ca...
Name: tags, dtype: object

In [377]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000,stop_words='english')

In [379]:
vector = cv.fit_transform(new_df['tags']).toarray()
vector.shape
from sklearn.metrics.pairwise import cosine_similarity
similarity = cosine_similarity(vector)
similarity

array([[1.        , 0.08964215, 0.06071767, ..., 0.02519763, 0.0277885 ,
        0.        ],
       [0.08964215, 1.        , 0.06350006, ..., 0.02635231, 0.        ,
        0.        ],
       [0.06071767, 0.06350006, 1.        , ..., 0.02677398, 0.        ,
        0.        ],
       ...,
       [0.02519763, 0.02635231, 0.02677398, ..., 1.        , 0.07352146,
        0.04774099],
       [0.0277885 , 0.        , 0.        , ..., 0.07352146, 1.        ,
        0.05264981],
       [0.        , 0.        , 0.        , ..., 0.04774099, 0.05264981,
        1.        ]])

In [381]:
new_df[new_df['title'] == 'The Lego Movie'].index[0]

744

In [386]:
def recommend(movie):
    index = new_df[new_df['title'] == movie].index[0]
    distances = sorted(list(enumerate(similarity[index])),reverse=True,key = lambda x: x[1])
    for i in distances[1:6]:
        print(new_df.iloc[i[0]].title)
        

In [387]:
recommend('Gandhi')


Gandhi, My Father
The Wind That Shakes the Barley
A Passage to India
Guiana 1838
Ramanujan


In [399]:
type(similarity)

numpy.ndarray

In [388]:
#pickle.dump(similarity,open('similarity.pkl','wb'))
pickle.dump(new_df,open('movie_list.pkl','wb'))


In [392]:
def compressed_pickle(title, data):
    with bz2.BZ2File(title + '.pbz2', 'w') as f:
        pickle.dump(data, f)
compressed_pickle('similarity',similarity)

In [403]:
import gzip
import numpy

f = gzip.GzipFile("similarity.npy.gz", "w")
numpy.save(file=f, arr=similarity)
f.close()